In [1]:
import csv
from tabulate import tabulate

# configure below to print tables for Jupyter notebook or for Latex
tablestyle = "fancy_grid"
# tablestyle = "latex"

def nano_to_milli(x):
    return x * 10**(-6)

error_margin = 0.03

In [2]:
header = ['alg','opt','dim','len','max','neg','time','unit','dev']
table_vals = []

with open('strat01b/data.csv', newline='') as csvfile:
    results_reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    ctr = 0
    for row in results_reader:
        #print(', '.join(row))
        if ctr > 0:
            table_vals.append(row)
        ctr += 1
        
#print(tabulate(row, headers=header, tablefmt=tablestyle))

In [3]:
def collect_pos_vals(header_val):
    lis_vals = []
    for row in table_vals:
        tmp = row[header.index(header_val)]
        if not tmp in lis_vals:
            lis_vals.append(tmp)
    return lis_vals
        
algs = collect_pos_vals('alg')
opts = collect_pos_vals('opt')
dims = collect_pos_vals('dim')
lens = collect_pos_vals('len')
size_or_length = collect_pos_vals('max')
neg_degrees = collect_pos_vals('neg')

In [4]:
def collect_corresponding_val(opt_val, algo, dupl_dim, fixed_size, neg_degree, nr_diff):
    for row in table_vals:
        test0 = row[header.index('opt')] == opt_val
        test1 = row[header.index('alg')] == algo
        test2 = row[header.index('dim')] == dupl_dim
        test3 = row[header.index('max')] == fixed_size
        test4 = row[header.index('neg')] == neg_degree
        test5 = row[header.index('len')] == nr_diff
        if test0 and test1 and test2 and test3 and test4 and test5:
            return int(row[header.index('time')])
    #print('not found')
    #print(opt_val, algo, dupl_dim, fixed_size, neg_degree, nr_diff)
    return 10**10

In [5]:
def compare_opts(algo, ran0, ran1):
    variable = neg_degrees[ran0:ran1]
    caption = "Comparison of different opts for " + algo + " for various negation degrees"
    print(caption)
    
    target_table = []
    max_factor = []
    for opt_val in opts:
        target_table.append([opt_val])
        max_factor.append(0)
    
    for nd in variable:
        min_opt = opts[0]
        min_val = 10**11
        for opt_val in opts:
            val = collect_corresponding_val(opt_val, algo, dims[0], size_or_length[0], nd, lens[0])
            if val < min_val:
                min_val = val
                min_opt = opt_val
            target_table[opts.index(opt_val)].append(val)
        for opt_val in opts:
            if opt_val == min_opt:
                target_table[opts.index(opt_val)].append('-')
            else:
                test_val = target_table[opts.index(opt_val)][-1]
                sf = round(test_val/min_val,3)
                target_table[opts.index(opt_val)].append(sf)
                if sf > max_factor[opts.index(opt_val)]:
                    max_factor[opts.index(opt_val)] = sf
            target_table[opts.index(opt_val)][-2] = nano_to_milli(target_table[opts.index(opt_val)][-2])
    
    header = ['Opt']
    for var in variable:
        header.append(var)
        header.append('SF')
                
    print(tabulate(target_table, headers=header, tablefmt=tablestyle))
    
    print("Max factors")
    for opt_val in opts:
        print(opt_val, max_factor[opts.index(opt_val)])

In [6]:
for algo in algs[1:]:
    compare_opts(algo, 0, 5)
    compare_opts(algo, 5, 10)
    compare_opts(algo, 10, len(neg_degrees))

Comparison of different opts for keygen for various negation degrees
╒═══════╤═════════╤═══════╤═════════╤═══════╤═════════╤═══════╤═════════╤═══════╤═════════╤═══════╕
│ Opt   │       1 │ SF    │       2 │ SF    │       3 │ SF    │       4 │ SF    │       5 │ SF    │
╞═══════╪═════════╪═══════╪═════════╪═══════╪═════════╪═══════╪═════════╪═══════╪═════════╪═══════╡
│ opt0  │ 211.328 │ 1.002 │ 330.893 │ 1.001 │ 450.088 │ 1.001 │ 569.71  │ 1.001 │ 688.762 │ 1.001 │
├───────┼─────────┼───────┼─────────┼───────┼─────────┼───────┼─────────┼───────┼─────────┼───────┤
│ opt1  │ 210.958 │ 1.001 │ 330.697 │ 1.0   │ 449.681 │ -     │ 569.351 │ 1.001 │ 688.349 │ 1.0   │
├───────┼─────────┼───────┼─────────┼───────┼─────────┼───────┼─────────┼───────┼─────────┼───────┤
│ opt2  │ 210.846 │ -     │ 330.569 │ -     │ 449.778 │ 1.0   │ 569.323 │ 1.0   │ 688.28  │ -     │
├───────┼─────────┼───────┼─────────┼───────┼─────────┼───────┼─────────┼───────┼─────────┼───────┤
│ opt3  │ 211.104 │ 1.001 │ 330